In [67]:
import numpy as np
import pandas as pd
import os
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

import re


In [68]:
rev_data = pd.read_csv('employee_reviews.csv')

rev_data.head()

topics = list(rev_data.columns.values)[10:15]

rev_data.head()

,Unnamed: 0,company,location,dates,job-title,summary,pros,cons,advice-to-mgmt,overall ratings,work balance,culture values,career opportunities,company benefits,senior management,helpful-count,link
0,1,google,none,"Dec 11, 2018",Current Employee - Anonymous Employee,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,none,5,4,5,5,4,5,0,https://www.glassdoor.com/Reviews/Google-Revie...
1,2,google,"Mountain View, CA","Jun 21, 2013",Former Employee - Program Manager,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,4,2,3,3,5,3,2094,https://www.glassdoor.com/Reviews/Google-Revie...
2,3,google,"New York, NY","May 10, 2014",Current Employee - Software Engineer III,Great balance between big-company security and...,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",Keep the focus on the user. Everything else wi...,5,5,4,5,5,4,949,https://www.glassdoor.com/Reviews/Google-Revie...
3,4,google,"Mountain View, CA","Feb 8, 2015",Current Employee - Anonymous Employee,The best place I've worked and also the most d...,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,Keep on NOT micromanaging - that is a huge ben...,5,2,5,5,4,5,498,https://www.glassdoor.com/Reviews/Google-Revie...
4,5,google,"Los Angeles, CA","Jul 19, 2018",Former Employee - Software Engineer,"Unique, one of a kind dream job",Google is a world of its own. At every other c...,"If you don't work in MTV (HQ), you will be giv...",Promote managers into management for their man...,5,5,5,5,5,5,49,https://www.glassdoor.com/Reviews/Google-Revie...


In [69]:
rev_data['Text'] = rev_data['company'].map(str)+ ' ' + rev_data['job-title'].map(str) + ' ' + rev_data['summary'].map(str) + ' ' +  rev_data['pros'].map(str) + ' ' + rev_data['cons'].map(str) + ' ' + rev_data['advice-to-mgmt'].map(str)



In [70]:
data = rev_data['Text'].values.tolist()

In [71]:
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

data = [re.sub("\-", "", sent) for sent in data]


In [72]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
        
data_words = list(sent_to_words(data))

print(data_words[:1])

[['google', 'current', 'employee', 'anonymous', 'employee', 'best', 'company', 'to', 'work', 'for', 'people', 'are', 'smart', 'and', 'friendly', 'bureaucracy', 'is', 'slowing', 'things', 'down', 'none']]


In [73]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['google', 'current', 'employee', 'anonymous', 'employee', 'best', 'company', 'to', 'work', 'for', 'people', 'are', 'smart', 'and', 'friendly', 'bureaucracy', 'is', 'slowing', 'things', 'down', 'none']


In [74]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [75]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['google', 'current', 'employee', 'anonymous', 'employee', 'good', 'company', 'work', 'people', 'smart', 'friendly', 'bureaucracy', 'slow', 'thing', 'none']]


In [76]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]]


In [77]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


In [79]:
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.039*"manager" + 0.035*"company" + 0.025*"team" + 0.020*"product" + 0.018*"many" + 0.017*"career" + 0.016*"people" + 0.015*"change" + 0.014*"senior" + 0.014*"opportunity"'), (1, '0.242*"microsoft" + 0.050*"management" + 0.028*"need" + 0.025*"review" + 0.023*"new" + 0.018*"system" + 0.017*"high" + 0.016*"support" + 0.012*"always" + 0.012*"performance"'), (2, '0.054*"people" + 0.044*"make" + 0.026*"really" + 0.025*"take" + 0.025*"thing" + 0.022*"well" + 0.018*"way" + 0.016*"much" + 0.016*"want" + 0.015*"also"'), (3, '0.078*"get" + 0.053*"not" + 0.045*"do" + 0.031*"time" + 0.027*"year" + 0.021*"go" + 0.021*"be" + 0.021*"job" + 0.015*"bad" + 0.014*"even"'), (4, '0.122*"employee" + 0.090*"work" + 0.066*"good" + 0.052*"great" + 0.049*"current" + 0.032*"none" + 0.030*"anonymous" + 0.026*"former" + 0.024*"company" + 0.022*"place"'), (5, '0.101*"customer" + 0.060*"service" + 0.052*"training" + 0.036*"meeting" + 0.030*"global" + 0.029*"home" + 0.029*"late" + 0.023*"test" + 0.015*"enjoy" +

In [80]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.1623098284682225

Coherence Score:  0.37998780959140355


In [81]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.270253 -0.043577       1        1  33.385944
0     -0.244730 -0.067042       2        1  24.189226
3      0.034593  0.143802       3        1  14.033418
1      0.244123 -0.358873       4        1  13.010602
2     -0.008281  0.099658       5        1  12.307920
5      0.244548  0.226033       6        1   3.072894, topic_info=     Category           Freq           Term          Total  loglift  logprob
368   Default   91607.000000      microsoft   91607.000000  30.0000  30.0000
4     Default  119115.000000       employee  119115.000000  29.0000  29.0000
13    Default   87851.000000           work   87851.000000  28.0000  28.0000
6     Default   64674.000000           good   64674.000000  27.0000  27.0000
95    Default   31837.000000            get   31837.000000  26.0000  26.0000
99    Default   50110.000000          great   50110.000000  25.0000  25.0000
3     Default   47208.000000        current   47208.000000  24.0000  24.0000
162   Default   21486.000000            not   21486.000000  23.0000  23.0000
9     Default   44142.000000         people   44142.000000  22.0000  22.0000
148   Default   27328.000000        manager   27328.000000  21.0000  21.0000
147   Default   18949.000000     management   18949.000000  20.0000  20.0000
73    Default   18225.000000             do   18225.000000  19.0000  19.0000
8     Default   31020.000000           none   31020.000000  18.0000  18.0000
361   Default   15941.000000           make   15941.000000  17.0000  17.0000
0     Default   29596.000000      anonymous   29596.000000  16.0000  16.0000
652   Default    9056.000000       customer    9056.000000  15.0000  15.0000
2     Default   48360.000000        company   48360.000000  14.0000  14.0000
93    Default   24875.000000         former   24875.000000  13.0000  13.0000
237   Default   18242.000000           time   18242.000000  12.0000  12.0000
433   Default   17699.000000           team   17699.000000  11.0000  11.0000
180   Default   21360.000000          place   21360.000000  10.0000  10.0000
33    Default   20515.000000        benefit   20515.000000   9.0000   9.0000
159   Default   10818.000000           need   10818.000000   8.0000   8.0000
457   Default   11620.000000           year   11620.000000   7.0000   7.0000
191   Default   14406.000000        product   14406.000000   6.0000   6.0000
300   Default   18090.000000       engineer   18090.000000   5.0000   5.0000
197   Default    9435.000000         really    9435.000000   4.0000   4.0000
423   Default   17399.000000       software   17399.000000   3.0000   3.0000
549   Default    9294.000000         review    9294.000000   2.0000   2.0000
12    Default    9032.000000          thing    9032.000000   1.0000   1.0000
...       ...            ...            ...            ...      ...      ...
212    Topic6    5346.610352        service    5347.551270   3.4824  -2.8180
930    Topic6    4660.668457       training    4661.609863   3.4823  -2.9553
1466   Topic6    3231.310547        meeting    3232.252441   3.4823  -3.3215
1413   Topic6    2693.119629         global    2694.060547   3.4822  -3.5037
1923   Topic6    2630.610107           home    2631.551025   3.4822  -3.5272
2190   Topic6    2580.058594           late    2581.002686   3.4822  -3.5466
923    Topic6    2058.607422           test    2059.549561   3.4821  -3.7724
80     Topic6    1377.375977          enjoy    1378.318359   3.4819  -4.1742
1155   Topic6    1337.310303      associate    1338.250732   3.4818  -4.2038
1693   Topic6    1287.645630  international    1288.588135   3.4818  -4.2416
1264   Topic6    1205.828735       creative    1206.769897   3.4818  -4.3073
1626   Topic6    1201.406128        willing    1202.349609   3.4818  -4.3109
1317   Topic6    1156.070312       schedule    1157.009644   3.4817  -4.3494
2544   Topic6    1129.273438        involve    1130.216675  